In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_columns = 0

In [2]:
train = pd.read_csv('./data/bulldozer/Train.csv')
train = train.sort_values(by='saledate', ascending=True)

C:\Users\Danny\AppData\Local\Temp\ipykernel_11600\3320590860.py:1: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('./data/bulldozer/Train.csv')


In [3]:
data_dict = pd.read_excel('./data/bulldozer/Data Dictionary.xlsx')
data_dict

f:\Coding\global-envs\ai-tink\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Variable,Description,Unnamed: 2
0,SalesID,unique identifier of a particular sale of a ...,NaN
1,MachineID,identifier for a particular machine; machin...,NaN
2,ModelID,identifier for a unique machine model (i.e. ...,NaN
3,datasource,source of the sale record; some sources are...,NaN
4,auctioneerID,"identifier of a particular auctioneer, i.e. ...",NaN
5,YearMade,year of manufacturer of the Machine,NaN
6,MachineHoursCurrentMeter,current usage of the machine in hours at tim...,NaN
7,UsageBand,"value (low, medium, high) calculated compari...",NaN
8,Saledate,time of sale,NaN
9,Saleprice,cost of sale in USD,NaN


In [4]:
stats = train.describe()
stats = stats.loc[:, ~stats.columns.str.endswith('ID')].drop('datasource', axis=1)
stats

,SalePrice,YearMade,MachineHoursCurrentMeter
count,401125.000000,401125.000000,1.427650e+05
mean,31099.712848,1899.156901,3.457955e+03
std,23036.898502,291.797469,2.759026e+04
min,4750.000000,1000.000000,0.000000e+00
25%,14500.000000,1985.000000,0.000000e+00
50%,24000.000000,1995.000000,0.000000e+00
75%,40000.000000,2000.000000,3.025000e+03
max,142000.000000,2013.000000,2.483300e+06


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401125 entries, 268962 to 22602
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   401125 non-null  int64  
 1   SalePrice                 401125 non-null  int64  
 2   MachineID                 401125 non-null  int64  
 3   ModelID                   401125 non-null  int64  
 4   datasource                401125 non-null  int64  
 5   auctioneerID              380989 non-null  float64
 6   YearMade                  401125 non-null  int64  
 7   MachineHoursCurrentMeter  142765 non-null  float64
 8   UsageBand                 69639 non-null   object 
 9   saledate                  401125 non-null  object 
 10  fiModelDesc               401125 non-null  object 
 11  fiBaseModel               401125 non-null  object 
 12  fiSecondaryDesc           263934 non-null  object 
 13  fiModelSeries             56908 non-null   ob

In [6]:
X_train = train.drop('SalePrice', axis=1)
y_train = train['SalePrice']

In [7]:
train.shape

(401125, 53)

In [8]:
cols_to_train = []
data_len = X_train.shape[0]

# len(train['auctioneerID'].loc[train['auctioneerID'].isnull()])
# if more than 20% of values are null, we can probably toss to start
nulls_threshold = 0.2
for col in X_train.columns:
    if col == 'Enclosure':
        pass
    num_null = len(X_train[col].loc[X_train[col].isnull()])
    null_ratio = num_null / data_len
    if null_ratio < nulls_threshold:
        cols_to_train.append(col)

print(len(cols_to_train))

14


In [9]:
'MachineHoursCurrentMeter' in cols_to_train

False

In [10]:
cols_to_train

['SalesID',
 'MachineID',
 'ModelID',
 'datasource',
 'auctioneerID',
 'YearMade',
 'saledate',
 'fiModelDesc',
 'fiBaseModel',
 'fiProductClassDesc',
 'state',
 'ProductGroup',
 'ProductGroupDesc',
 'Enclosure']

In [11]:
X_train_subset = (X_train[cols_to_train]
    .drop(['SalesID', 'MachineID', 'ModelID', 'saledate'], axis=1)
)

In [12]:
X_train_subset.head()

,datasource,auctioneerID,YearMade,fiModelDesc,fiBaseModel,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Enclosure
268962,132,2.0,1998,751,751,Skid Steer Loader - 1251.0 to 1351.0 Lb Operat...,Pennsylvania,SSL,Skid Steer Loaders,OROPS
237812,132,2.0,1000,PC40,PC40,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Pennsylvania,TEX,Track Excavators,EROPS
121285,132,2.0,1993,312,312,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Pennsylvania,TEX,Track Excavators,EROPS w AC
221519,132,2.0,1994,EX120-3,EX120,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Pennsylvania,TEX,Track Excavators,EROPS
283617,132,2.0,1994,B502B,B502,"Hydraulic Excavator, Track - 4.0 to 5.0 Metric...",Pennsylvania,TEX,Track Excavators,EROPS


In [13]:
X_train_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401125 entries, 268962 to 22602
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   datasource          401125 non-null  int64  
 1   auctioneerID        380989 non-null  float64
 2   YearMade            401125 non-null  int64  
 3   fiModelDesc         401125 non-null  object 
 4   fiBaseModel         401125 non-null  object 
 5   fiProductClassDesc  401125 non-null  object 
 6   state               401125 non-null  object 
 7   ProductGroup        401125 non-null  object 
 8   ProductGroupDesc    401125 non-null  object 
 9   Enclosure           400800 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 33.7+ MB


In [14]:
# Split out a validation dataset

valid_ratio = 0.2
train_len = round(len(X_train_subset) * (1-valid_ratio))

X_train_subset_final = X_train_subset.iloc[:train_len, :]
y_train_final = y_train.iloc[:train_len]

X_valid = X_train_subset.iloc[train_len+1:, :]
y_valid = y_train.iloc[train_len+1:]

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor

In [16]:
categorical_string_features = [
    'fiModelDesc',
    'fiBaseModel', 
    'fiProductClassDesc', 
    'state', 
    'ProductGroup', 
    'ProductGroupDesc', 
    'Enclosure'
]
categorical_string_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

categorical_num_features = [
    'datasource', 
    'auctioneerID', 
]
categorical_num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])

numerical_features = ['YearMade']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat_str', categorical_string_transformer, categorical_string_features),
        ('cat_num', categorical_num_transformer, categorical_num_features),
        ('num', numerical_transformer, numerical_features)
    ],
    verbose=True
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(verbose=3, n_jobs=-1))
])

In [17]:
# categorical_transformer. (categorical_features)
model.fit(X_train_subset_final, y_train_final)
model.score(X_valid, y_valid)

[ColumnTransformer] ....... (1 of 3) Processing cat_str, total=   0.7s
[ColumnTransformer] ....... (2 of 3) Processing cat_num, total=   0.1s
[ColumnTransformer] ........... (3 of 3) Processing num, total=   0.0s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 15.3min


building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 60.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.8s finished


0.7844089070514768

In [19]:
model.score(X_train_subset_final, y_train_final)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   14.4s finished


0.9420848268322899

In [20]:
preds = model.predict(X_valid)
preds

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    3.9s finished


array([28259.35714286, 31152.83333333, 31087.4978355 , ...,
       75056.        , 37661.52380952, 27376.125     ])

In [22]:
preds_train = model.predict(X_train_subset_final)
preds_train

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    9.3s finished


array([ 8151.31302521, 10793.62698413, 32030.28225941, ...,
       42528.33333333, 14884.70238095, 10574.85714286])

In [31]:
from sklearn.metrics import mean_squared_log_error

mean_squared_log_error(y_valid, preds, squared=False)

0.2972812340816371

In [30]:
mean_squared_log_error(y_train_final, preds_train, squared=False)

0.1603467959449414

In [18]:
import pickle

file_name = 'model-initial.pkl'
pickle.dump(model, open(file_name, 'wb'))

In [32]:
# Public leaderboard submission simulation

test = pd.read_csv('./data/bulldozer/Valid.csv')
test_solution = pd.read_csv('./data/bulldozer/ValidSolution.csv')

X_test = (test[cols_to_train]
    .drop(['SalesID', 'MachineID', 'ModelID', 'saledate'], axis=1)
)
y_test = test_solution['SalePrice']

preds_test = model.predict(X_test)

# Baseline gets me 0.26835
# That would put me at 66th in the final public leaderboard
# But that uses the test data? Not sure - either way, pretty good!
round(
    mean_squared_log_error(y_test, preds_test, squared=False), # type: ignore
    5
)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


0.26835

In [38]:
# Training random forest takes 1 hour, try iterating on simpler model,
# then extend updates to random forest?

from sklearn import linear_model

model_ridge = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', linear_model.Ridge(alpha=0.2))
])
model_ridge.fit(X_train_subset_final, y_train_final)

mean_squared_log_error(
    y_valid,
    np.abs(model_ridge.predict(X_valid)),
    squared=False
)

[ColumnTransformer] ....... (1 of 3) Processing cat_str, total=   0.8s
[ColumnTransformer] ....... (2 of 3) Processing cat_num, total=   0.1s
[ColumnTransformer] ........... (3 of 3) Processing num, total=   0.0s


0.37616509428883066

In [39]:
from sklearn import svm

model_svm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', svm.SVR(kernel='linear'))
])
model_svm.fit(X_train_subset_final, y_train_final)

mean_squared_log_error(
    y_valid,
    np.abs(model_svm.predict(X_valid)),
    squared=False
)

[ColumnTransformer] ....... (1 of 3) Processing cat_str, total=   0.8s
[ColumnTransformer] ....... (2 of 3) Processing cat_num, total=   0.1s
[ColumnTransformer] ........... (3 of 3) Processing num, total=   0.0s
